## lidando com falta de dados
---

In [2]:
import numpy as np
import pandas as pd

para dados faltando, o pandas usa tanto o valor `None` próprio do python, que é um tipo especial de número flutuante (float); bem como, pode usar `np.nan`, um valor especial do numpy (not a number).

a primeiro opção serve apenas para preencher buracos do objeto pandas, series ou dataframe. Desta forma, esse objeto não conseguirá, por exemplo, passar por qualquer tipo de operação, como `.sum()` ou `.min()`, assim como, outras operações que consiga passar, serão feitos no nível do python, tornando a operação lenta.

In [3]:
a = np.array([1, None, 3.5, True])
a

array([1, None, 3.5, True], dtype=object)

já, a segunda opção é mais rápida já que é nativo do numpy e suporta outros tipos de operações, mesmo que estas só resultem em `NaN`

In [4]:
a = np.array([1, np.nan, 3.5, True])
a

array([1. , nan, 3.5, 1. ])

In [5]:
1 + np.nan

nan

In [6]:
a.min()

nan

In [7]:
a.max()

nan

para resolver isto, numpy oferece métodos específicos:

In [9]:
np.nanmin(a)

1.0

In [10]:
np.nanmax(a)

3.5

observe que `NaN` só funciona com operações que suportam tipo `float`; qualquer outro tipo - `string`, `integer`, `bool` - não funciona.

o pandas lida com esses valores de forma intercabiável:

In [11]:
b = pd.Series([2, np.nan, 3, None])
b

0    2.0
1    NaN
2    3.0
3    NaN
dtype: float64

o panda também muda o tipo do objeto para acomodar esses valores especiais:

In [16]:
c = pd.Series([2, 3])
c.dtype

dtype('int64')

In [17]:
c[0] = None
c.dtype

dtype('float64')

#### funções para valores nulos
---

df.função()|operação
---|---
isnull|retorna False ou True indicando falta de dados
notnull|o inverso de isnull()
dropna|retorna o objeto panda sem valores nulos, se este tiver
fillna(<tipo>)|retorna o objeto panda substituindo o valor nulo pelo passado

como os objetos pandas suportam fancing indexing, estes métodos podem ser usados para encontrar valores específicos:

In [19]:
c[c.isnull()]

0   NaN
dtype: float64

enquanto que para series o resultado do método `dropna()` é simples, para dataframes, este método ignorará todas as linhas que tenham valores nulos. para que este ignore as colunas, é preciso passar como parâmetro `axis=1` ou `axis='columns'`

In [23]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [24]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [25]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


o problema aqui fica o fato de se perder informações, com isto, pode, ainda, ser passado o parâmetro `how=`, que recebe os valores `any`, fazendo com a função funcione exatamento como ela já funciona, e `all`, que fará com que ela só ignore o eixo, dependendo do que foi passado em `axis`, se este for completamento preenchido com valores nulos:

In [26]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [27]:
df.dropna(axis=0, how='any')

,0,1,2,3


In [28]:
df.dropna(axis=0, how='all')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [29]:
df.dropna(axis=1, how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


pode ainda ser passado o parâmetro `thresh=` que recebe um valor mínimo de valores nulos para o eixo permança:

In [32]:
df.dropna(thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


o método `fillna()` recebe qualquer valor que deseja substituir os valores nulos:

In [34]:
df.fillna(-1)

,0,1,2,3
0,1.0,-1.0,2,-1.0
1,2.0,3.0,5,-1.0
2,-1.0,4.0,6,-1.0


pode, ainda, receber o parâmetro `method=` que, se receber `ffill`, preenche o valor nulo com o valor imediatamento anterior a ele:

In [37]:
df.fillna(method='ffill')

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [38]:
df.fillna(axis=1, method='ffill')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


ou, o parâmetro `method=` receber `bfill`, preenchendo os valores nulos com os valores que o imediatamente segue:

In [39]:
df.fillna(axis=1, method='bfill')

,0,1,2,3
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,NaN
